In [1]:
from keras.models import Sequential
from keras.layers import Dense,RepeatVector,Dropout
from keras.layers import Flatten
#from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import pandas as pd
import numpy as np

In [2]:
from sklearn.preprocessing import MinMaxScaler


In [3]:
data = pd.read_csv('CNA.csv')


In [4]:
data

,Date,Open,High,Low,Close,Adj Close,Volume
0,1973-02-21,0.000000,5.666667,5.500000,5.500000,3.902713,45900
1,1973-02-22,0.000000,5.583333,5.500000,5.541667,3.932280,48900
2,1973-02-23,0.000000,5.625000,5.375000,5.375000,3.814015,957900
3,1973-02-26,0.000000,5.458333,5.333333,5.458333,3.873148,54900
4,1973-02-27,0.000000,5.500000,5.375000,5.416667,3.843580,44700
...,...,...,...,...,...,...,...
11879,2020-03-26,31.120001,32.770000,30.620001,32.619999,32.619999,260000
11880,2020-03-27,31.600000,32.830002,29.840000,31.990000,31.990000,313400
11881,2020-03-30,32.029999,33.240002,31.150000,32.980000,32.980000,248500
11882,2020-03-31,32.720001,32.720001,30.879999,31.040001,31.040001,267700


In [5]:
data = data.filter(['Close'])


In [6]:
# Lấy 100 dòng cuối cùng của df
data = data.tail(1000)
data

,Close
10884,31.990000
10885,31.790001
10886,32.099998
10887,32.099998
10888,32.299999
...,...
11879,32.619999
11880,31.990000
11881,32.980000
11882,31.040001


In [9]:
#Tiền xử lý
data.fillna(0, inplace=True)

# Normalize data
scaler = MinMaxScaler()
dataScale = scaler.fit_transform(data['Close'].values.reshape(-1,1))

In [10]:
dataScale

array([[0.17210466],
       [0.16493367],
       [0.17604868],
       [0.17604868],
       [0.18321974],
       [0.17820004],
       [0.15130868],
       [0.16529217],
       [0.16565072],
       [0.17246322],
       [0.17067049],
       [0.16708496],
       [0.15812118],
       [0.20258155],
       [0.19290064],
       [0.18931518],
       [0.18572959],
       [0.18752232],
       [0.18608821],
       [0.19541049],
       [0.1817855 ],
       [0.18823944],
       [0.17569019],
       [0.17963428],
       [0.16780208],
       [0.18644671],
       [0.18070989],
       [0.1964861 ],
       [0.195052  ],
       [0.21082821],
       [0.20544989],
       [0.19863746],
       [0.20652564],
       [0.20329867],
       [0.20078882],
       [0.19720322],
       [0.18895655],
       [0.18572959],
       [0.18142701],
       [0.1817855 ],
       [0.17891716],
       [0.1767658 ],
       [0.14879883],
       [0.12692717],
       [0.12477588],
       [0.12405876],
       [0.12585156],
       [0.134

In [11]:
#Chia dữ liệu thành train test
data = pd.DataFrame(dataScale)

#dataScale = data
train_percent = 0.7
train_size = int(len(data) * train_percent)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

In [12]:
#Chia dữ liệu thành train test
data = train_data

#dataScale = data
train_percent = 0.7
train_size = int(len(data) * train_percent)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

In [13]:
#Chuẩn bị dữ liệu để đưa vào mô hình
def prepare_data(data, size_window,size_predict,stepWindow):
    X, y = [], []
    startWindow = 0
    for i in range(len(data) - size_window - 1):
        if (len(data[(startWindow + size_window):(startWindow + size_window + size_predict) , 0]) != size_predict):
            break
        X.append(data[startWindow:(startWindow + size_window), :])
        y.append(data[(startWindow + size_window):(startWindow + size_window + size_predict) , 0])
          
       
        startWindow += stepWindow
    return np.array(X), np.array(y)


size_window = 90
size_predict = 1
stepWindow = 1
X_train, y_train = prepare_data(train_data.values, size_window, size_predict, stepWindow)
X_test, y_test = prepare_data(test_data.values, size_window, size_predict, stepWindow)

In [ ]:
import time
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

# Khởi tạo một số tham số
input_shape = (90, 1)
batch_size = 16
epochs = 10000
num_filters = [128] # các số lượng filter khác nhau [32, 64, 128, 256]
kernel_sizes = [16] # các kích thước kernel khác nhau [2,3,4,6,8,16,32]
scores = [] # list để lưu kết quả đánh giá mô hình


# Load dữ liệu
# Thực hiện tìm kiếm lưới
for num_filter in num_filters:
    for kernel_size in kernel_sizes:
        # Khởi tạo mô hình với số lớp convolution và kích thước kernel tương ứng
        model = Sequential()
        model.add(Conv1D(filters=num_filter, kernel_size=kernel_size, activation='relu', input_shape=input_shape))
        model.add(Conv1D(filters=num_filter, kernel_size=kernel_size, activation='relu'))
        model.add(Conv1D(filters=num_filter, kernel_size=kernel_size, activation='relu'))


        #MaxPooling là một phép lọc giá trị lớn nhất từ các kết quả của lớp convolution trước 
        model.add(MaxPooling1D(pool_size=2))
        #Đưa về dạng 1 chiều 
        model.add(Flatten())
        model.add(Dense(100, activation='relu'))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')
        # Lưu thời điểm bắt đầu huấn luyện
        start_time = time.time()
        # Huấn luyện mô hình
        model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,validation_data=(X_test, y_test))
        # Đánh giá kết quả của mô hình và lưu vào scores
        y_pred = model.predict(X_test)
        y_pred = np.reshape(y_pred, (-1, 1))
        # Lưu thời điểm kết thúc huấn luyện
        end_time = time.time()
        # Tính thời gian huấn luyện
        training_time = end_time - start_time
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mape = (mean_absolute_percentage_error(y_test, y_pred))*100
        
        scores.append((num_filter, kernel_size, mse, rmse, mape, training_time))
        
# Hiển thị kết quả tìm kiếm lưới
for score in scores:
    print("num_filters=%d, kernel_size=%d, mse=%.5f, rmse=%.5f, mape=%.5f, training_time=%.2f seconds" % (score[0], score[1], score[2], score[3], score[4],score[5]))



Epoch 1/10000
35/35 [==============================] - 3s 54ms/step - loss: 0.0197 - val_loss: 0.0053
Epoch 2/10000
35/35 [==============================] - 2s 52ms/step - loss: 0.0102 - val_loss: 0.0096
Epoch 3/10000
35/35 [==============================] - 2s 53ms/step - loss: 0.0066 - val_loss: 0.0065
Epoch 4/10000
35/35 [==============================] - 2s 50ms/step - loss: 0.0060 - val_loss: 0.0073
Epoch 5/10000
35/35 [==============================] - 2s 50ms/step - loss: 0.0045 - val_loss: 0.0063
Epoch 6/10000
35/35 [==============================] - 2s 52ms/step - loss: 0.0037 - val_loss: 0.0034
Epoch 7/10000
35/35 [==============================] - 2s 53ms/step - loss: 0.0032 - val_loss: 0.0072
Epoch 8/10000
35/35 [==============================] - 2s 51ms/step - loss: 0.0030 - val_loss: 0.0027
Epoch 9/10000
35/35 [==============================] - 2s 52ms/step - loss: 0.0025 - val_loss: 0.0103
Epoch 10/10000
35/35 [==============================] - 2s 51ms/step - loss: 0.002

35/35 [==============================] - 2s 49ms/step - loss: 8.9691e-04 - val_loss: 0.0034
Epoch 81/10000
35/35 [==============================] - 2s 49ms/step - loss: 6.5182e-04 - val_loss: 0.0038
Epoch 82/10000
35/35 [==============================] - 2s 49ms/step - loss: 8.0552e-04 - val_loss: 0.0032
Epoch 83/10000
35/35 [==============================] - 2s 49ms/step - loss: 8.1321e-04 - val_loss: 0.0032
Epoch 84/10000
35/35 [==============================] - 2s 49ms/step - loss: 7.7118e-04 - val_loss: 0.0030
Epoch 85/10000
35/35 [==============================] - 2s 49ms/step - loss: 7.8940e-04 - val_loss: 0.0041
Epoch 86/10000
35/35 [==============================] - 2s 49ms/step - loss: 8.2370e-04 - val_loss: 0.0034
Epoch 87/10000
35/35 [==============================] - 2s 49ms/step - loss: 6.7745e-04 - val_loss: 0.0044
Epoch 88/10000
35/35 [==============================] - 2s 49ms/step - loss: 0.0012 - val_loss: 0.0034
Epoch 89/10000
35/35 [==============================] - 

35/35 [==============================] - 2s 48ms/step - loss: 1.4374e-04 - val_loss: 0.0042
Epoch 233/10000
35/35 [==============================] - 2s 49ms/step - loss: 1.6426e-04 - val_loss: 0.0032
Epoch 234/10000
35/35 [==============================] - 2s 49ms/step - loss: 1.7560e-04 - val_loss: 0.0039
Epoch 235/10000
35/35 [==============================] - 2s 49ms/step - loss: 3.2801e-04 - val_loss: 0.0032
Epoch 236/10000
35/35 [==============================] - 2s 49ms/step - loss: 2.1206e-04 - val_loss: 0.0031
Epoch 237/10000
35/35 [==============================] - 2s 48ms/step - loss: 1.8165e-04 - val_loss: 0.0047
Epoch 238/10000
35/35 [==============================] - 2s 48ms/step - loss: 2.5341e-04 - val_loss: 0.0031
Epoch 239/10000
35/35 [==============================] - 2s 48ms/step - loss: 1.5390e-04 - val_loss: 0.0029
Epoch 240/10000
35/35 [==============================] - 2s 48ms/step - loss: 1.6901e-04 - val_loss: 0.0031
Epoch 241/10000
35/35 [=====================

35/35 [==============================] - 2s 49ms/step - loss: 1.3579e-04 - val_loss: 0.0037
Epoch 309/10000
35/35 [==============================] - 2s 48ms/step - loss: 1.5385e-04 - val_loss: 0.0027
Epoch 310/10000
35/35 [==============================] - 2s 48ms/step - loss: 1.9355e-04 - val_loss: 0.0024
Epoch 311/10000
35/35 [==============================] - 2s 48ms/step - loss: 1.6486e-04 - val_loss: 0.0024
Epoch 312/10000
35/35 [==============================] - 2s 48ms/step - loss: 1.2388e-04 - val_loss: 0.0032
Epoch 313/10000
35/35 [==============================] - 2s 49ms/step - loss: 1.1537e-04 - val_loss: 0.0029
Epoch 314/10000
35/35 [==============================] - 2s 48ms/step - loss: 1.1880e-04 - val_loss: 0.0027
Epoch 315/10000
35/35 [==============================] - 2s 48ms/step - loss: 1.7068e-04 - val_loss: 0.0027
Epoch 316/10000
35/35 [==============================] - 2s 48ms/step - loss: 1.3808e-04 - val_loss: 0.0030
Epoch 317/10000
35/35 [=====================

35/35 [==============================] - 2s 49ms/step - loss: 1.9604e-04 - val_loss: 0.0049
Epoch 385/10000
35/35 [==============================] - 2s 49ms/step - loss: 2.6844e-04 - val_loss: 0.0027
Epoch 386/10000
35/35 [==============================] - 2s 49ms/step - loss: 1.4087e-04 - val_loss: 0.0022
Epoch 387/10000
35/35 [==============================] - 2s 48ms/step - loss: 8.7998e-05 - val_loss: 0.0027
Epoch 388/10000
35/35 [==============================] - 2s 49ms/step - loss: 9.9790e-05 - val_loss: 0.0024
Epoch 389/10000
35/35 [==============================] - 2s 48ms/step - loss: 6.9623e-05 - val_loss: 0.0034
Epoch 390/10000
35/35 [==============================] - 2s 48ms/step - loss: 9.5675e-05 - val_loss: 0.0032
Epoch 391/10000
35/35 [==============================] - 2s 48ms/step - loss: 7.4467e-05 - val_loss: 0.0033
Epoch 392/10000
35/35 [==============================] - 2s 49ms/step - loss: 1.0149e-04 - val_loss: 0.0034
Epoch 393/10000
35/35 [=====================

...

35/35 [==============================] - 2s 49ms/step - loss: 3.1390e-05 - val_loss: 0.0030
Epoch 1593/10000
35/35 [==============================] - 2s 49ms/step - loss: 3.4158e-05 - val_loss: 0.0030
Epoch 1594/10000
35/35 [==============================] - 2s 49ms/step - loss: 7.4129e-05 - val_loss: 0.0029
Epoch 1595/10000
35/35 [==============================] - 2s 49ms/step - loss: 4.4511e-05 - val_loss: 0.0029
Epoch 1596/10000
35/35 [==============================] - 2s 49ms/step - loss: 2.6140e-05 - val_loss: 0.0031
Epoch 1597/10000
35/35 [==============================] - 2s 49ms/step - loss: 1.4032e-04 - val_loss: 0.0039
Epoch 1598/10000
35/35 [==============================] - 2s 49ms/step - loss: 9.6142e-05 - val_loss: 0.0025
Epoch 1599/10000
35/35 [==============================] - 2s 49ms/step - loss: 1.0069e-04 - val_loss: 0.0056
Epoch 1600/10000
35/35 [==============================] - 2s 49ms/step - loss: 8.7372e-05 - val_loss: 0.0029
Epoch 1601/10000
35/35 [============

35/35 [==============================] - 2s 49ms/step - loss: 6.7975e-06 - val_loss: 0.0025
Epoch 1743/10000
35/35 [==============================] - 2s 49ms/step - loss: 7.4191e-06 - val_loss: 0.0025
Epoch 1744/10000
35/35 [==============================] - 2s 49ms/step - loss: 7.7583e-06 - val_loss: 0.0024
Epoch 1745/10000
35/35 [==============================] - 2s 49ms/step - loss: 1.3809e-05 - val_loss: 0.0024
Epoch 1746/10000
35/35 [==============================] - 2s 49ms/step - loss: 1.8680e-05 - val_loss: 0.0025
Epoch 1747/10000
35/35 [==============================] - 2s 49ms/step - loss: 1.9248e-05 - val_loss: 0.0031
Epoch 1748/10000
35/35 [==============================] - 2s 49ms/step - loss: 2.2745e-05 - val_loss: 0.0026
Epoch 1749/10000
35/35 [==============================] - 2s 49ms/step - loss: 1.6983e-05 - val_loss: 0.0024
Epoch 1750/10000
35/35 [==============================] - 2s 49ms/step - loss: 1.3407e-05 - val_loss: 0.0026
Epoch 1751/10000
35/35 [============

35/35 [==============================] - 2s 51ms/step - loss: 1.4500e-05 - val_loss: 0.0024
Epoch 1893/10000
35/35 [==============================] - 2s 49ms/step - loss: 3.0865e-05 - val_loss: 0.0034
Epoch 1894/10000
35/35 [==============================] - 2s 50ms/step - loss: 1.2000e-04 - val_loss: 0.0054
Epoch 1895/10000
35/35 [==============================] - 2s 49ms/step - loss: 2.8228e-04 - val_loss: 0.0073
Epoch 1896/10000
35/35 [==============================] - 2s 49ms/step - loss: 2.0350e-04 - val_loss: 0.0047
Epoch 1897/10000
35/35 [==============================] - 2s 49ms/step - loss: 1.3350e-04 - val_loss: 0.0034
Epoch 1898/10000
35/35 [==============================] - 2s 49ms/step - loss: 3.5991e-05 - val_loss: 0.0026
Epoch 1899/10000
35/35 [==============================] - 2s 48ms/step - loss: 2.2413e-05 - val_loss: 0.0032
Epoch 1900/10000
35/35 [==============================] - 2s 49ms/step - loss: 1.0124e-05 - val_loss: 0.0027
Epoch 1901/10000
35/35 [============

35/35 [==============================] - 2s 49ms/step - loss: 1.2240e-06 - val_loss: 0.0025
Epoch 2043/10000
35/35 [==============================] - 2s 49ms/step - loss: 1.3218e-06 - val_loss: 0.0026
Epoch 2044/10000
35/35 [==============================] - 2s 48ms/step - loss: 1.1108e-06 - val_loss: 0.0026
Epoch 2045/10000
35/35 [==============================] - 2s 50ms/step - loss: 1.2752e-06 - val_loss: 0.0025
Epoch 2046/10000
35/35 [==============================] - 2s 50ms/step - loss: 1.2906e-06 - val_loss: 0.0026
Epoch 2047/10000
35/35 [==============================] - 2s 50ms/step - loss: 1.2891e-06 - val_loss: 0.0024
Epoch 2048/10000
35/35 [==============================] - 2s 49ms/step - loss: 1.3670e-06 - val_loss: 0.0026
Epoch 2049/10000
35/35 [==============================] - 2s 49ms/step - loss: 2.8345e-06 - val_loss: 0.0025
Epoch 2050/10000
35/35 [==============================] - 2s 49ms/step - loss: 4.4298e-06 - val_loss: 0.0024
Epoch 2051/10000
35/35 [============

35/35 [==============================] - 2s 49ms/step - loss: 5.2930e-06 - val_loss: 0.0022
Epoch 2193/10000
35/35 [==============================] - 2s 49ms/step - loss: 3.1622e-06 - val_loss: 0.0021
Epoch 2194/10000
35/35 [==============================] - 2s 48ms/step - loss: 2.7789e-06 - val_loss: 0.0022
Epoch 2195/10000
35/35 [==============================] - 2s 49ms/step - loss: 1.9536e-06 - val_loss: 0.0022
Epoch 2196/10000
35/35 [==============================] - 2s 49ms/step - loss: 2.0332e-06 - val_loss: 0.0022
Epoch 2197/10000
35/35 [==============================] - 2s 48ms/step - loss: 2.0507e-06 - val_loss: 0.0023
Epoch 2198/10000
35/35 [==============================] - 2s 49ms/step - loss: 1.5120e-06 - val_loss: 0.0022
Epoch 2199/10000
35/35 [==============================] - 2s 50ms/step - loss: 1.4096e-06 - val_loss: 0.0022
Epoch 2200/10000
35/35 [==============================] - 2s 49ms/step - loss: 1.1577e-06 - val_loss: 0.0023
Epoch 2201/10000
35/35 [============

In [ ]:
data